## Scikit-Learn Preprocessing and Training Pipeline
##### from sklearn.feature_extraction.text import TfidfVectorizer
##### from sklearn.naive_bayes import MultinomialNB
### Using data from Google Cloud Storage and DWC

## Install fedml_gcp package

In [ ]:
pip install fedml_gcp

## Import Libraries

In [ ]:
from fedml_gcp import DwcGCP
import numpy as np
import pandas as pd

## Create DwcGCP Instance to access class methods and train model

It is expected that the bucket name passed here already exists in Cloud Storage.

In [ ]:
dwc = DwcGCP(project_name='example-project',
                 bucket_name='<bucket-name>')

### Create tar bundle of script folder so GCP can use it for training

Before running this cell, please ensure that the script package has all the necessary files for a training job.

In [ ]:
dwc.make_tar_bundle('PreprocessedPipeline.tar.gz', 
                    'PreprocessingAndTrainingPipeline', 
                    'preprocessed-pipeline/train/PreprocessedPipeline.tar.gz')


### Train Model

GCP takes in training inputs that are specific to the training job and the environment needed.

In the training inputs, we are the python module. This is the module that your script package is named, and it references the task.py file inside the script package.

We are also passing args which hold the table name to get data from. Before running the following cell, you should have a config.json file in the script package with the specified values to allow you to access to DWC.

You should also have the follow view `IMDB_TEST_VIEW` created in your DWC. To gather this data, please refer to https://www.kaggle.com/mantri7/imdb-movie-reviews-dataset?select=train_data+%281%29.csv and download the test dataset.

This script also downloads data from Cloud Storage and uses it for training. Please download the train dataset from the link below and upload it to your Cloud Storage bucket (file path = <bucket-name>/data) before proceeding. https://www.kaggle.com/mantri7/imdb-movie-reviews-dataset?select=train_data+%281%29.csv

In [ ]:
training_inputs = {
    'scaleTier': 'BASIC',
    'packageUris': ['gs://<bucket-name>/preprocessed-pipeline/train/PreprocessedPipeline.tar.gz'],
    'pythonModule': 'trainer.task',
    'args': ['--table_name', 'IMDB_TEST_VIEW',
             '--table_size', '1',
            '--file_path', 'data/imdb_train.csv',
            '--bucket_name', '<bucket-name>'],
    'region': 'us-east1',
    'jobDir': 'gs://<bucket-name>',
    'runtimeVersion': '2.5',
    'pythonVersion': '3.7',
    'scheduling': {'maxWaitTime': '3600s', 'maxRunningTime': '7200s'}
}

In [ ]:
dwc.train_model('preprocessed_pipeline_final', training_inputs)

### Deploy model

In [ ]:
dwc.deploy(model_name='<example-model>', 
           model_location='/preprocessed-pipeline/model', version='v1', region='us-east1')